In [11]:
import numpy as np
import json
import pickle
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import geopandas as gpd
from shapely.geometry import Polygon

import time as T

In [2]:
def load_road_mask(MASK_PATH):
    # Load Road Mask
    ROAD_MASK = True
    if ROAD_MASK:
        # Road Mask
        road = pickle.load(open(MASK_PATH, "rb"))
        road_ = np.pad(road, pad_width=((8, 9), (6, 6)))
    return road_

def interpolate_coords(attr_shape):
    # Bounding box to interpolate coordinates
    yMin = 52.359 - 0.009
    yMax = 52.854 + 0.008
    xMin = 13.189 - 0.006
    xMax = 13.625 + 0.006

#     xtranslate = (xMax-xMin)/attr_shape[-1]
#     ytranslate = (yMax-yMin)/attr_shape[-2]
#     x = np.linspace(xMin,xMax,attr_shape[-1])-xtranslate # Lon
#     y = np.linspace(yMin,yMax,attr_shape[-2])[::-1]-ytranslate # Lat

    x = np.linspace(xMin,xMax,attr_shape[-1]) # Lon
    y = np.linspace(yMin,yMax,attr_shape[-2])[::-1] # Lat

    xv, yv = np.meshgrid(x,y,indexing='xy')
    xv = np.round(xv,3)
    yv = np.round(yv,3)
    
    return xv, yv

# Split attribution by time epoch and volume/speed/incident level
def spatial_attribution_agg(attr):
    '''
    An spatial attribution aggregate by time epoch and volume/speed/incident level
    return: np.array.shape = [12,3,512,448] 12 time epochs, 3 represents volume/speed/incident level at the time epoch
    '''
    attr_all = np.zeros((12,3,attr.shape[-2],attr.shape[-1]))
    for i in range(12):
        start_epoch = i
        # Non-static features per time epoch
        x = attr[0, start_epoch*9:(start_epoch+1)*9, :, :] 

        # Agg speed/volume per direction
        volume_idx = np.arange(0, 8, 2)
        speed_idx = np.arange(1, 8, 2)
        volume = np.mean(x[volume_idx, :, :], axis=0)[None,...]
        speed = np.mean(x[speed_idx, :, :], axis=0)[None,...]
        # Incident per time epoch
        incident = x[-1, :, :][None,...]
        # Attribution for this Time epoch
        attr_all[i] = np.concatenate((volume,speed,incident))[None,...]
    return attr_all

def spatial_attribution_agg_flat(attr):
    '''
    An spatial attribution aggregate by time epoch and volume/speed/incident level, and static features
    return: np.array.shape = [43,512,448] = [C,H,W]; last 7 in dim=0 are static features
    '''
    attr_ts = spatial_attribution_agg(attr)
    attr_ts = attr_ts.reshape(-1, attr_ts.shape[-2], attr_ts.shape[-1])
    static = attr[0, -7:, :, :]
    attr_all = np.concatenate((attr_ts,static))
    return attr_all




def zoom_to_range(window, window_size, zoom_window_num, arr_shape):
    '''
        Zoom to a range
    '''
    # Zoom to the range
    zoom_window_num_y = zoom_window_num + 2
    start_x = (window[0]-zoom_window_num)*window_size
    end_x = (window[0]+zoom_window_num)*window_size
    start_y = (window[1]-zoom_window_num_y)*window_size
    end_y = (window[1]+zoom_window_num_y)*window_size
    start_x = start_x if start_x>0 else 0
    end_x = end_x if end_x<arr_shape[-2] else arr_shape[-2]
    start_y = start_y if start_x>0 else 0
    end_y = end_y if end_y<arr_shape[-1] else arr_shape[-1]
    return (start_x, end_x, start_y, end_y)



## Save as geojson polygons

In [27]:
log_root = r"C:\Users\jingyli\OwnDrive\IPA\attribution_Result\unet\attribution_pickle\resUnet"
MASK_PATH = r"C:\Users\jingyli\OwnDrive\IPA\python-eda-code\utils\Berlin.mask"
output_root = r"C:\Users\jingyli\OwnDrive\IPA\web_json"
window_size = 21

def save_window_to_geojson(window, date, time):
    print(f"Processing: w{window}-{date}-{time}")
    file_path = f"{date}_berlin_9ch{time}-saliency-target-channel0-W{window[0]}-{window[1]}.npy"
    file1_path = f"{date}_berlin_9ch{time}-saliency-target-channel1-W{window[0]}-{window[1]}.npy"
    # OUTPUT PATH
    output_path = os.path.join(output_root, f"{date}_{time}")
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    output_filename = os.path.join(output_path, f"{date}_{time}_W{window[0]}-{window[1]}.geo.json")
    # Test if file existis
    if os.path.isfile(output_filename):
        return

    # Attribution to volume / speed
    attr = np.load(os.path.join(log_root, f"{date}_{time}", file_path))
    attr1 = np.load(os.path.join(log_root, f"{date}_{time}", file1_path))

    # Aggregate attr
    attr_all = spatial_attribution_agg_flat(attr)
    attr_all1 = spatial_attribution_agg_flat(attr1)

    ## Add up volume/speed
    attr_all = np.concatenate([attr_all,attr_all1])
    print(attr_all.shape)

    # Zoom to a small area
    zoom_window_num = 2
    (start_x, end_x, start_y, end_y) = zoom_to_range(window, window_size, zoom_window_num, attr_all.shape)
    print(f"zoom to [{start_x}:{end_x}, {start_y}:{end_y}]")
    attr_zoom = attr_all[:, 
                         start_x : end_x,
                         start_y : end_y]
    print(attr_zoom.shape)

    # Load Road mask
    road_ = load_road_mask(MASK_PATH)

    road_zoom = road_[start_x : end_x,
                         start_y : end_y]
    road_zoom = road_zoom.astype(np.int8)
    print(road_zoom.shape)

    # Interpolate Coords
    xv, yv = interpolate_coords((1, 115, 512, 448))

    # Apply road mask; 
    xv_zoom = xv[start_x : end_x,
                         start_y : end_y]
    yv_zoom = yv[start_x : end_x,
                         start_y : end_y]
    mask = road_zoom ==1
    output = np.concatenate([xv_zoom[mask][:,None],
                             yv_zoom[mask][:,None],
                            ], axis=1)
    output_attr = attr_zoom[:,mask].transpose()
    # Loop through grids and create polygons
    polygons = []
    for rec in output:
        x_left = rec[0]
        y_up = rec[1]
        x_right = x_left + 0.001
        y_bot = y_up - 0.001
        attr = rec[2:]
        polygon = Polygon([(x_left,y_up), (x_right, y_up), (x_right,y_bot),(x_left, y_bot)])
        polygons.append(polygon)

    # Create geo dataframe
    gdf_data = {'geometry':polygons}
    for i in range(output_attr.shape[1]):
        gdf_data[i] = output_attr[:,i]
    grid = gpd.GeoDataFrame(gdf_data,crs="EPSG:4326")

    st = T.time()
    # Save
    data = grid.to_json(show_bbox=False, na="drop")
    with open(output_filename,"w") as f:
        f.writelines(data)
    print(f"Saving used {T.time()-st} s")
    

def save_stable_to_geojson(date, time, stable_name):
    print(f"Processing: {stable_name}-{date}-{time}")
    # prediction and error
    result_path = f"{date}_berlin_9ch{time}-{stable_name}.npy"
    result = np.load(os.path.join(log_root, f"{date}_{time}", result_path))
    
    # OUTPUT PATH
    output_path = os.path.join(output_root, f"{date}_{time}")
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    output_result_path = os.path.join(output_path, f"{date}_{time}_{stable_name}.geo.json")
    
    # Test if file existis
    if os.path.isfile(output_result_path):
        return
    
    # Load Road mask
    road_ = load_road_mask(MASK_PATH).astype(np.int8)

    # Interpolate Coords
    xv, yv = interpolate_coords((1, 115, 512, 448))


    mask = road_ ==1
    output = np.concatenate([xv[mask][:,None],
                             yv[mask][:,None],
                            ], axis=1)
    output_result = result[:,mask].transpose()

    # Loop through grids and create polygons
    polygons = []
    for rec in output:
        x_left = rec[0]
        y_up = rec[1]
        x_right = x_left + 0.001
        y_bot = y_up - 0.001
        attr = rec[2:]
        polygon = Polygon([(x_left,y_up), (x_right, y_up), (x_right,y_bot),(x_left, y_bot)])
        polygons.append(polygon)

    # Create geo dataframe
    gdf_data = {'geometry':polygons}
    for i in range(output_result.shape[1]):
        gdf_data[i] = output_result[:,i]
    grid = gpd.GeoDataFrame(gdf_data,crs="EPSG:4326")

    st = T.time()
    # Save
    data = grid.to_json(show_bbox=False, na="drop")
    with open(output_result_path,"w") as f:
        f.writelines(data)
    print(f"Saving used {T.time()-st} s")
    
    
# times = [84,132,204] 
# dates = ["2019-09-19","2019-10-19"]
dates = ["2019-08-10"]
times = [84]

for time in times:
    for date in dates:
#         time = 84
#         date = "2019-09-19"

        save_stable_to_geojson(date, time, "err-pred")
        save_stable_to_geojson(date, time, "gt-inc")

        window_centers = [[12,12],] # [y,x][16,4],[14,5]
        window_num = 1
        for window in window_centers:  
#             y = np.arange(window[0]-window_num, window[0]+window_num+1)
#             x = np.arange(window[1]-window_num, window[1]+window_num+1)
            y = [window[0]]
            x = [window[1]]
            for y_ in y:
                for x_ in x:
                    save_window_to_geojson([y_,x_], date, time)

Processing: err-pred-2019-08-10-84
Processing: gt-inc-2019-08-10-84
Processing: w[12, 12]-2019-08-10-84
(86, 512, 448)
zoom to [210:294, 168:336]
(86, 84, 168)
(84, 168)
Saving used 11.641252517700195 s


v_pred,
s_pred,
v_gt_epoch-v_pred,
s_gt_epoch-s_pred

In [13]:
# prediction and error
result_path = f"{date}_berlin_9ch{time}-err-pred.npy"
result = np.load(os.path.join(log_root, f"{date}_{time}", result_path))
output_result_path =  os.path.join(output_root, f"{date}_{time}", f"{date}_{time}_err-pred.geo.json")

# Load Road mask
road_ = load_road_mask(MASK_PATH).astype(np.int8)

# Interpolate Coords
xv, yv = interpolate_coords((1, 115, 512, 448))


mask = road_ ==1
output = np.concatenate([xv[mask][:,None],
                         yv[mask][:,None],
                        ], axis=1)
output_result = result[:,mask].transpose()

# Loop through grids and create polygons
polygons = []
for rec in output:
    x_left = rec[0]
    y_up = rec[1]
    x_right = x_left + 0.001
    y_bot = y_up - 0.001
    attr = rec[2:]
    polygon = Polygon([(x_left,y_up), (x_right, y_up), (x_right,y_bot),(x_left, y_bot)])
    polygons.append(polygon)

# Create geo dataframe
gdf_data = {'geometry':polygons}
for i in range(output_result.shape[1]):
    gdf_data[i] = output_result[:,i]
grid = gpd.GeoDataFrame(gdf_data,crs="EPSG:4326")

st = T.time()
# Save
data = grid.to_json(show_bbox=True, na="drop")
with open(output_result_path,"w") as f:
    f.writelines(data)
print(f"Saving used {T.time()-st} s")

Saving used 24.51560664176941 s


In [52]:
# incident in input
date = "2019-08-30"
time = "75"
result_path = f"{date}_berlin_9ch{time}-gt-inc.npy"
result = np.load(os.path.join(log_root, f"{date}_{time}", result_path))

for i in range(12):
    result[i,262:277,259:268] = 45
#     result[i,263,254] = 45


output_result_path =  os.path.join(output_root, f"{date}_{time}", f"{date}_{time}_gt-inc.geo.json")

# Load Road mask
road_ = load_road_mask(MASK_PATH).astype(np.int8)

# Interpolate Coords
xv, yv = interpolate_coords((1, 115, 512, 448))


mask = road_ ==1
output = np.concatenate([xv[mask][:,None],
                         yv[mask][:,None],
                        ], axis=1)
output_result = result[:,mask].transpose()

# Loop through grids and create polygons
polygons = []
for rec in output:
    x_left = rec[0]
    y_up = rec[1]
    x_right = x_left + 0.001
    y_bot = y_up - 0.001
    attr = rec[2:]
    polygon = Polygon([(x_left,y_up), (x_right, y_up), (x_right,y_bot),(x_left, y_bot)])
    polygons.append(polygon)

# Create geo dataframe
gdf_data = {'geometry':polygons}
for i in range(output_result.shape[1]):
    gdf_data[i] = output_result[:,i]
grid = gpd.GeoDataFrame(gdf_data,crs="EPSG:4326")

st = T.time()
# Save
data = grid.to_json(show_bbox=True, na="drop")
with open(output_result_path,"w") as f:
    f.writelines(data)
print(f"Saving used {T.time()-st} s")

Saving used 25.5755033493042 s


In [51]:
result[:,262,259]

array([45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45., 45.],
      dtype=float32)

## Save as json

In [20]:
log_root = r"C:\Users\jingyli\OwnDrive\IPA\attribution_Result\unet\attribution_pickle\resUnet"
MASK_PATH = r"C:\Users\jingyli\OwnDrive\IPA\python-eda-code\utils\Berlin.mask"
output_root = r"C:\Users\jingyli\OwnDrive\IPA\web_json"
window_size = 21


time = 84
date = "2019-09-19"


window = [16, 8]  # [y,x]
file_path = f"{date}_berlin_9ch{time}-saliency-target-channel0-W{window[0]}-{window[1]}.npy"
file1_path = f"{date}_berlin_9ch{time}-saliency-target-channel1-W{window[0]}-{window[1]}.npy"
# OUTPUT PATH
output_path = os.path.join(output_root, f"{date}_{time}")
if not os.path.exists(output_path):
    os.mkdir(output_path)
output_filename = os.path.join(output_path, f"{date}_{time}_W{window[0]}-{window[1]}.json")


# Attribution to volume / speed
attr = np.load(os.path.join(log_root, f"{date}_{time}", file_path))
attr1 = np.load(os.path.join(log_root, f"{date}_{time}", file1_path))

# Aggregate attr
attr_all = spatial_attribution_agg_flat(attr)
attr_all1 = spatial_attribution_agg_flat(attr1)

## Add up volume/speed
attr_all = np.concatenate([attr_all,attr_all1])
print(attr_all.shape)

# Zoom to a small area
zoom_window_num = 2
(start_x, end_x, start_y, end_y) = zoom_to_range(window, window_size, zoom_window_num, attr_all.shape)
print(f"zoom to [{start_x}:{end_x}, {start_y}:{end_y}]")
attr_zoom = attr_all[:, 
                     start_x : end_x,
                     start_y : end_y]
print(attr_zoom.shape)

# Load Road mask
road_ = load_road_mask(MASK_PATH)

road_zoom = road_[start_x : end_x,
                     start_y : end_y]
road_zoom = road_zoom.astype(np.int8)
print(road_zoom.shape)

# Interpolate Coords
xv, yv = interpolate_coords((1, 115, 512, 448))

# Apply road mask; Save road attribution to json
xv_zoom = xv[start_x : end_x,
                     start_y : end_y]
yv_zoom = yv[start_x : end_x,
                     start_y : end_y]
mask = road_zoom ==1
output = np.concatenate([xv_zoom[mask][:,None],
                         yv_zoom[mask][:,None],
                         attr_zoom[:,mask].transpose()
                        ], axis=1)
output_json = json.dumps(output.tolist())

with open(output_filename,"w") as f:
    f.writelines(output_json)

(86, 512, 448)
zoom to [294:378, 84:252]
(86, 84, 168)
(84, 168)


v_pred,
s_pred,
v_gt_epoch-v_pred,
s_gt_epoch-s_pred

In [27]:
# prediction and error
result_path = f"{date}_berlin_9ch{time}-err-pred.npy"
result = np.load(os.path.join(log_root, f"{date}_{time}", result_path))
output_result_path =  os.path.join(log_root, f"{date}_{time}", f"{date}_{time}_pred-err.json")

# Load Road mask
road_ = load_road_mask(MASK_PATH).astype(np.int8)

# Interpolate Coords
xv, yv = interpolate_coords((1, 115, 512, 448))


mask = road_ ==1
output = np.concatenate([xv[mask][:,None],
                         yv[mask][:,None],
                         result[:,mask].transpose()
                        ], axis=1)

output_json = json.dumps(output.tolist())

with open(output_result_path,"w") as f:
    f.writelines(output_json)

In [8]:
# incident in input
result_path = f"{date}_berlin_9ch{time}-gt-inc.npy"
result = np.load(os.path.join(log_root, f"{date}_{time}", result_path))
output_result_path =  os.path.join(log_root, f"{date}_{time}", f"{date}_{time}_gt-inc.json")

# Load Road mask
road_ = load_road_mask(MASK_PATH).astype(np.int8)

# Interpolate Coords
xv, yv = interpolate_coords((1, 115, 512, 448))


mask = road_ ==1
output = np.concatenate([xv[mask][:,None],
                         yv[mask][:,None],
                         result[:,mask].transpose()
                        ], axis=1)

output_json = json.dumps(output.tolist())

with open(output_result_path,"w") as f:
    f.writelines(output_json)

In [11]:
result[result!=0]

array([0.5882353 , 0.5882353 , 0.5882353 , ..., 0.19607843, 1.        ,
       0.39215687], dtype=float32)